In [1]:
import pandas as pd
import numpy as np
import sklearn
from scipy.stats import norm
from sklearn.preprocessing import StandardScaler
from scipy import stats
import warnings
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
warnings.filterwarnings('ignore')

In [2]:
train = pd.read_csv("/Users/Triveni/Desktop/dataScience/data/train.csv")

In [3]:
catVars = [col for col in list(train) if train[col].dtype=="object"]
trainPlusDummies = pd.get_dummies(data=train,columns=catVars)
trainPlusDummies.dropna(inplace=True)

In [4]:
from sklearn.feature_selection import RFECV
from sklearn.linear_model import LassoCV
from sklearn.model_selection import cross_val_score

In [5]:
X = trainPlusDummies.drop(columns=["SalePrice","Id"])
Y = trainPlusDummies.SalePrice
featureNames = list(X)
x_train,x_test,y_train,y_test = sklearn.model_selection.train_test_split(X,Y)

In [13]:
lasso = LassoCV()
rfe = RFECV(estimator=lasso,cv=5)
rfe.fit(x_train, y_train)
rfeFeatures = X.columns[rfe.get_support()]

In [11]:
rfeFeatures

Index(['MSSubClass', 'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1',
       'TotalBsmtSF', 'GrLivArea', 'GarageArea', 'WoodDeckSF', 'OpenPorchSF',
       'ScreenPorch', 'PoolArea'],
      dtype='object')

In [8]:
x_train_new = rfe.transform(x_train)
x_test_new = rfe.transform(x_test)

In [58]:
linearModel = sklearn.linear_model.LinearRegression()
linearModel.fit(x_train_new,y_train)
rfeScores = cross_val_score(linearModel,x_test_new,y_test,cv=10)

In [60]:
rfeScores.mean()

0.55505753127311586

In [19]:
from sklearn.ensemble import RandomForestRegressor
from numpy.random import RandomState

In [46]:
R = np.random.RandomState(3)
rf = RandomForestRegressor(random_state = 3,max_features='sqrt')
rf.fit(x_train,y_train)
print("Features sorted by their score:",
sorted(zip(map(lambda x: float(round(x, 4)), rf.feature_importances_), featureNames), 
             reverse=True)[:10])

Features sorted by their score: [(0.1315, 'OverallQual'), (0.0605, '1stFlrSF'), (0.0586, 'BsmtFinSF1'), (0.0525, 'GarageArea'), (0.0522, 'GrLivArea'), (0.0504, 'GarageCars'), (0.0479, 'TotalBsmtSF'), (0.047, 'FullBath'), (0.0394, 'BsmtQual_Ex'), (0.033, 'KitchenQual_TA')]


In [62]:
rfScores = cross_val_score(rf,x_test_new,y_test,cv=10)

In [61]:
rfScores.mean()

0.64383293349685577